In this project, you will be tasked with predicting the 'Status' of flights based on historical flight data.

In [ ]:
# You might use this block to download the training data
import os
import gdown

if not os.path.exists("flights-data.tgz"):
    data_url = "https://drive.google.com/file/d/10d1zrylGKQQp_Sl-5ejuphn4Q1vKDIo8/view?usp=share_link"
    gdown.download(data_url, fuzzy=True)
    !tar -xvzf "flights-data.tgz"

Now, you need to implement your training code

Then, it's time to predict and submit the results.

In [ ]:
import os
import gdown
import numpy as np
import pandas as pd

# input your student ID and name here
student_id = "50012962"
student_name = "Bowen_LIU"

if not os.path.exists("test_data.csv"):
    data_url = "https://drive.google.com/file/d/1eLZRSP9zb9KkdeRg-8CGuak3a_xLS6YF/view?usp=drive_link"
    gdown.download(data_url, fuzzy=True)
data = pd.read_csv("test_data.csv")

# WARNING: Change this line to your prediction code!!! Do not submit this random result!!!
data["Status"] = np.random.choice(np.array(["normal", "cancel", "delay"]), size=len(data), replace=True)

# Submit this ipynb and the csv file, you will be graded according to the f1 score (macro) and the quality of your code
data.to_csv(f"{student_id}_{student_name}.csv")